In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

4 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = None
train_size  = 45000
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28, 1)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
batch_size = 256
epochs = 200

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
# shuffle
seed = 0
x_train_all, y_train_all = shaffle(x_train_all, y_train_all, seed)

In [6]:
# down sample
x_train = x_train_all[:train_size]
y_train = y_train_all[:train_size]

x_valid = x_train_all[train_size:]
y_valid = y_train_all[train_size:]

x_test = x_test_all[:test_size]
y_test = y_test_all[:test_size]

In [7]:
x_train, x_valid, x_test = x_train.reshape((-1, *image_shape)), x_valid.reshape((-1, *image_shape)), x_test.reshape((-1, *image_shape))

In [8]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(
    lambda image, label: (tf.image.random_flip_left_right(image), label)
).map(
    lambda image, label: (tf.image.resize_with_crop_or_pad(image, image_shape[0]+6, image_shape[1]+6), label)
).map(
    lambda image, label: (tf.image.random_crop(image, (image_shape[0], image_shape[1], image_shape[2])), label)
).map(
    lambda image, label: (tf.image.random_contrast(image, lower=0.2, upper=1.0), label)
).shuffle(
    100000
).batch(
    batch_size
).prefetch(10)

In [9]:
valid_ds = tf.data.Dataset.from_tensor_slices((x_valid, y_valid)).batch(batch_size)

In [10]:
layers = tf.keras.layers

In [11]:
class BasicBlock(tf.keras.layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.downsample = tf.keras.Sequential()
            self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                       kernel_size=(1, 1),
                                                       strides=stride))
            self.downsample.add(tf.keras.layers.BatchNormalization())
        else:
            self.downsample = lambda x: x

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

def make_basic_block_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BasicBlock(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BasicBlock(filter_num, stride=1))

    return res_block

In [12]:
class ResNetTypeI(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeI, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=16,
                                            kernel_size=(3, 3),
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.layer1 = make_basic_block_layer(filter_num=16,
                                             blocks=layer_params[0])
        self.layer2 = make_basic_block_layer(filter_num=32,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_basic_block_layer(filter_num=64,
                                             blocks=layer_params[2],
                                             stride=2)
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(10)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output

In [13]:
model = ResNetTypeI([5, 5, 5])

In [14]:
model.build(input_shape=(batch_size,*image_shape))

In [15]:
model.summary()

Model: "res_net_type_i"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  448       
_________________________________________________________________
batch_normalization (BatchNo multiple                  64        
_________________________________________________________________
sequential (Sequential)      (256, 32, 32, 16)         23840     
_________________________________________________________________
sequential_1 (Sequential)    (256, 16, 16, 32)         89824     
_________________________________________________________________
sequential_3 (Sequential)    (256, 8, 8, 64)           355776    
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense (Dense)                multiple               

In [16]:
def scheduler(epoch, lr):
    if epoch < 10:
        return 1e-3
    elif epoch < 90:
        return 1e-2
    elif epoch < 150:
        return 1e-3
    else:
        return 1e-4

In [17]:
model.compile(optimizer=tf.keras.optimizers.SGD(momentum=0.9),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [18]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [21]:
model.fit(x=train_ds, validation_data=valid_ds, epochs=epochs, callbacks=[callback])

Epoch 1/200
176/176 [==============================] - 27s 152ms/step - loss: 2.1149 - accuracy: 0.2270 - val_loss: 2.2247 - val_accuracy: 0.1564
Epoch 2/200
176/176 [==============================] - 14s 77ms/step - loss: 1.8595 - accuracy: 0.3116 - val_loss: 1.8745 - val_accuracy: 0.3112
Epoch 3/200
176/176 [==============================] - 25s 142ms/step - loss: 1.7580 - accuracy: 0.3472 - val_loss: 2.0402 - val_accuracy: 0.2644
Epoch 4/200
176/176 [==============================] - 13s 76ms/step - loss: 1.6891 - accuracy: 0.3729 - val_loss: 1.7577 - val_accuracy: 0.3434
Epoch 5/200
176/176 [==============================] - 13s 76ms/step - loss: 1.6376 - accuracy: 0.3941 - val_loss: 1.6380 - val_accuracy: 0.3972
Epoch 6/200
176/176 [==============================] - 13s 77ms/step - loss: 1.5801 - accuracy: 0.4211 - val_loss: 1.6468 - val_accuracy: 0.4076
Epoch 7/200
176/176 [==============================] - 13s 76ms/step - loss: 1.5302 - accuracy: 0.4397 - val_loss: 1.5326 - val_

176/176 [==============================] - 14s 78ms/step - loss: 0.0474 - accuracy: 0.9855 - val_loss: 0.5388 - val_accuracy: 0.8716
Epoch 116/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0481 - accuracy: 0.9850 - val_loss: 0.5244 - val_accuracy: 0.8758
Epoch 117/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0456 - accuracy: 0.9860 - val_loss: 0.5227 - val_accuracy: 0.8766
Epoch 118/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0461 - accuracy: 0.9855 - val_loss: 0.5227 - val_accuracy: 0.8798
Epoch 119/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0437 - accuracy: 0.9868 - val_loss: 0.5358 - val_accuracy: 0.8746
Epoch 120/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0435 - accuracy: 0.9864 - val_loss: 0.5276 - val_accuracy: 0.8764
Epoch 121/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0432 - accuracy: 0.9865 - val_loss: 0.5287 - val_ac

Epoch 171/200
176/176 [==============================] - 14s 77ms/step - loss: 0.0319 - accuracy: 0.9905 - val_loss: 0.5625 - val_accuracy: 0.8778
Epoch 172/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0326 - accuracy: 0.9901 - val_loss: 0.5635 - val_accuracy: 0.8770
Epoch 173/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0322 - accuracy: 0.9908 - val_loss: 0.5638 - val_accuracy: 0.8764
Epoch 174/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0329 - accuracy: 0.9907 - val_loss: 0.5642 - val_accuracy: 0.8766
Epoch 175/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0317 - accuracy: 0.9905 - val_loss: 0.5659 - val_accuracy: 0.8762
Epoch 176/200
176/176 [==============================] - 14s 78ms/step - loss: 0.0325 - accuracy: 0.9906 - val_loss: 0.5641 - val_accuracy: 0.8764
Epoch 177/200
176/176 [==============================] - 14s 79ms/step - loss: 0.0324 - accuracy: 0.9904 - val_loss: 0

In [19]:
model.load_weights('./model_weights/resnet32_train=all_ce.h5')

In [21]:
model.evaluate(x=x_test, y=y_test_all, verbose=0)

[0.5649635086476803, 0.87200004]

In [24]:
# model.save_weights('./model_weights/resnet32_train=all_ce.h5')

In [ ]:
# with pooling, acc can reach 72.2%
# w.o. pooling, acc is at most 67%

In [22]:
tmp = onp.load('./npy/cifar-eps-time-any-npy/cifar-fgsm-eps-0.03-time-None.npy')
print("==========NTK============")
model.evaluate(tmp, y_test[:128], verbose=0)

==========NTK============


[1.4168568849563599, 0.734375]

In [23]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-ce.npy')
print("==========CE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========CE============


[0.7829372922424227, 0.8383789]

In [24]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=4096-mse.npy')
print("==========MSE============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========MSE============


[0.9031526042381302, 0.8071289]

In [26]:
tmp = onp.load('./npy/cifar-fgsm-eps-0.03-time-None-nngp.npy')
model.evaluate(tmp, y_test[:128], verbose=0)

[1.589853584766388, 0.7109375]

In [21]:
tmp = onp.load('./batch_NTK_simple.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[0.8903837203979492, 0.81201171875]

In [22]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-train=all-ce.npy')
print("==========small============")
model.evaluate(tmp, y_test[:2048], verbose=0)

==========small============


[1.136065125465393, 0.77587890625]

In [23]:
tmp = onp.load('./batch_NTK_cnn_19.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[0.8085411190986633, 0.828125]

In [24]:
tmp = onp.load('./npy/cifar-untargeted-cifar-nn-grey-box-cnn19-train=all-ce.npy')
model.evaluate(tmp, y_test[:2048], verbose=0)

[1.027262568473816, 0.7900390625]